In [1]:
using Plots, LinearAlgebra

# HW 3 Solutions: Numerical Solutions of Classical Equations of Motion
### Gabe Schumm
### Due October 18th, 2021

## Simulation Functions 

In [2]:
#Data structure containing moon info
mutable struct Moon 
    m::Float64 #mass in kg
    r::Float64 #radius of orbit in meters
    T::Float64 #period in seconds

    x::Array{Float64} #position vector
    v::Array{Float64} #velocity vector

    α::Int64 #angle w.r.t. the equitorial plane in DEGREES

end
 
#Generator of Moon object
function Moon(α::Int64)
    
    m = 7.3477e22 
    r = 384400e3
    T = 27.25 * 86164 
    
    x =  zeros(Float64, 3) 
    v =  zeros(Float64, 3) 
    
    Moon(m, r, T, x, v, α)
end

#Data structure containing satellite info
mutable struct Satellite

    x::Vector{Float64} #position vector
    v::Vector{Float64} #velocity vector
    
    last_phi::Float64 #store last phi value to determine when to increment n_r
    n_r::Int64 #number of revolutions around the earth

end

#Generator of Satellite object
function Satellite()
    
    x =  zeros(Float64, 3)
    v =  zeros(Float64, 3)
    
    Satellite(x, v, 0, 0)
end

#Data structure containing info for simulation
mutable struct System
    G::Float64 #gravity constant
    M_e::Float64 #mass of earth
    T_s::Int64 #sidereal day in seconds

    moon::Moon #Moon object used in simulation
    sat::Satellite #Satellite object used in simulation
    
    a::Float64 #initial orbit parameter
    
    t::Float64 #instaneous time of simulation
    dt::Float64 #time step
    t_max::Float64 #total total of simulation

    N_w::Int64 #cadence of data writing for simulation

    outfile::String #data output filename

end




 

#Generator of System object
#N_t = fraction of sidereal day used for dt
#N_max = number of days to integrate over
function System(α, a, N_t, N_max, N_w, outfile=false) 
    G = 6.6743e-11 # m^3 / kg^2
    M_e = 5.9736e24 # kg
    T_s = 86164 # seconds
    
    dt = T_s/N_t
    t_max = N_max * T_s
    
    
    moon = Moon(α)
    sat = Satellite()
    

    if !isdir(string("data/alpha_", α))
        mkdir(string("data/alpha_", α))
    end
    
    outfile = string("data/alpha_", α,"/sat_a", a, "Nt", N_t, ".dat")
    
    
    System(G, M_e, T_s, moon, sat, a, 0, dt, t_max, N_w, outfile)
end



System

In [3]:
#Returns acceleration of satellite at current time
function get_accel(sys)
    moon = sys.moon
    sat = sys.sat
    
    r_s = norm(sat.x)

    x_sm = sat.x .- moon.x
    r_sm = norm(x_sm)
    
    return (-(sys.G * sys.M_e) * sat.x / r_s^3) - ((sys.G * moon.m) * x_sm / r_sm^3)  #3-element vector
end


#Initialize moon position and satellite position.velocity
function init_system(sys)
    moon, sat = sys.moon, sys.sat
    
    #set moon position according to provided equation
    moon.x = [moon.r * cosd(moon.α), 0, moon.r * sind(moon.α)]
    
    #r_geo and v_geo according to provided equation
    #if a≠1, the intitial orbit is not the geo-stationary orbit
    r_geo = ((sys.G * sys.M_e * (sys.a*sys.T_s)^2) / (4 * pi^2)) ^(1/3)
    v_geo = sqrt(sys.G * sys.M_e / r_geo)
    
    #set satellite position 
    sat.x =  [r_geo, 0, 0]
    #calculate satellite acceleration at t=0 
    accel_0 = get_accel(sys)
    #integrate backwards half a time step to calculate v_(-1/2) for leapfrog algorithm
    sat.v = [0,v_geo ,0] .- (0.5 * sys.dt) * accel_0

end

#Update moon position according to current time
function update_moon!(sys)
    moon = sys.moon
    
    t_n = sys.t
    
    moon.x = [moon.r * cosd(moon.α) * cos(2 * pi * t_n/moon.T),
        moon.r * sin(2 * pi * t_n/moon.T),
        moon.r * sind(moon.α) * cos(2 * pi * t_n/moon.T)]
end

#Leapfrog integration algorithm
function leapfrog!(sys)
    sat = sys.sat
    
    accel = get_accel(sys) #first calculate acceleration a_n
    sat.v .+= sys.dt * accel #advance velocity to get v_(n+1/2)
    sat.x .+= sys.dt * sat.v #advance position to get x_n+1
end

#Update full system one time step
function update_sys!(sys)
    sys.t += sys.dt #increase system time by dt
    update_moon!(sys) #update moon position using equation of motion
    leapfrog!(sys) #update satellite position using leapfrog algorithm
end

#Calculate relevant data and write it to file f
function write_data(sys, f)
    t = sys.t #column 1
    phi = mod(atan(sys.sat.x[2], sys.sat.x[1]), 2*pi) #converts from [-π, π] to [0, 2π], column 2
    r = norm(sys.sat.x) #in METERS column 3
    
    #phi_(n+1) - phi_n will always be positive,
    #unless phi_(n+1) is greater than 2π aka in the first qudrant of the x-y plane
    #if this difference in negative, then the satellite will have just completed a revolution
    #and n_r in incremented by one
    if phi - sys.sat.last_phi<0  
        sys.sat.n_r += 1
    end
    
    #2π*n_r + phi gives cumulative phi
    #2π t/T_s gives expected cumulative phi
    delta_phi = (2*pi * sys.sat.n_r) + phi -  (2*pi  * sys.t / sys.T_s) #column 4
    
    
    delta_r = r - ((sys.G * sys.M_e * sys.T_s^2) / (4 * pi^2)) ^(1/3) #column 5
    
    #polar angle theta in spherical coordinates, measured from z-axis
    #π/2 - theta gives deviation from equitorial plane 
    Theta = pi/2 - mod(acos(sys.sat.x[3]/r), 2*pi) #column 6
    
    sys.sat.last_phi = phi #set current phi to last_phi for next run
    
    #write data to provided file f
    println(f, t, " ", phi, " ", r, " ", delta_phi, " ", delta_r, " ", Theta)
  
end

#Run simulation with provided parameters in sys
function run_simulation(sys)
    open(sys.outfile, "w") do f #open outfile as f
        for i = 0:(sys.t_max/sys.dt -1) #loop over number of integration steps necesary to reach prodivded t_max
            if mod(i, sys.N_w) == 0 #if integration step is multiple of N_w, write relevant data to outfile f
                write_data(sys, f)
            end
            update_sys!(sys) #update system at each integration step
        end
    end
end

run_simulation (generic function with 1 method)

## $\alpha = 0$

### $a=1$

In [4]:
sys = System(0, 1, 10^4, 100, 10^2); #α, a, N_t for dt, N_max for t_max, N_w for data writing

#Begin by initializing system (moon and satellite pos/velo)
init_system(sys)
#Then run simulation
#run_simulation(sys)

3-element Vector{Float64}:
    0.9658300777510512
 3074.9068178570305
    0.0

In [6]:
sys.sat.x

3-element Vector{Float64}:
 4.2167508691982634e7
 0.0
 0.0

### Testing different $a$ Values

In [4]:
sys_a = System(0, 1.000485, 10^4, 500, 10^2);
init_system(sys_a)
run_simulation(sys_a)

LoadError: UndefVarError: norm not defined

## $\alpha = 25$

### $a=1$

In [9]:
sys = System(25, 1, 10^4, 500, 10^2);
init_system(sys)
run_simulation(sys);

### Testing $a$ Values

In [ ]:
sys = System(25, .5, 10^4, 500, 10^2);
init_system(sys)
run_simulation(sys);

### Initializing satellite with negative $z$ velocity

In [54]:

sys = System(25, 1.000438, 10^4, 100, 10^2);

moon, sat = sys.moon, sys.sat

moon.x = [moon.r * cosd(moon.α) * cos(2 * pi * sys.t/moon.T),
    moon.r * sin(2 * pi * sys.t/moon.T),
    moon.r * sind(moon.α) * cos(2 * pi * sys.t/moon.T)]


r_geo = ((sys.G * sys.M_e * (sys.a*sys.T_s)^2) / (4 * pi^2)) ^(1/3)
v_geo = sqrt(sys.G * sys.M_e / r_geo)

sat.x =  [r_geo, 0, 0]
accel_0 = get_accel(sys)
sat.v = [0,v_geo - v_geo*1e-7 ,-sqrt(v_geo^2-(v_geo - v_geo*1e-7)^2)] .- (0.5 * sys.dt) * accel_0;

open("data/test_zvel.dat", "w") do f
        for i = 0:(sys.t_max/sys.dt -1)
            if mod(i, sys.N_w) == 0
                write_data(sys, f)
            end
            update_sys!(sys)
        end
    end